In [2]:
# importing the requests library
import requests
import json
import csv
import pandas as pd

In [ ]:
# api-endpoints
athlete_url = "http://www.strava.com/api/v3/athlete/"
athlete_zones_url = "http://www.strava.com/api/v3/athlete/zones"
athlete_stats_url = "http://www.strava.com/api/v3/athletes/" + str(athlete['id']) + "/stats"
activities_url = "http://www.strava.com/api/v3/athlete/activities"
activity_url = "http://www.strava.com/api/v3/activities/" + <activity_id>
activity_zone_url = "http://www.strava.com/api/v3/activities/" + <activity_id> + "/zones"

In [3]:
to_be_posted = 'https://www.strava.com/oauth/authorize?client_id=115408&redirect_uri=http://localhost/exchangetoken&response_type=code&scope=activity:read_all,profile:read_all'

# start here after oauth
client_secret = 'aa1e167a4cf8f4b7047f9783090a705a8bcbea7d'
authorizationCode = '6569d50bd57c2d80c5a8ceb169563669ae39a4d2' # update here

post_url = f'http://www.strava.com/oauth/token?client_id=115408&client_secret={client_secret}&code={authorizationCode}&grant_type=authorization_code'

r = requests.post(url = post_url)

access_token = r.json()['access_token']
refresh_token = r.json()['refresh_token']

print(f'Access_token: {access_token}')
print(f'Refresh_token: {refresh_token}')

Access_token: ab15181dd0a777978c82fa17393d5419f8c3b412
Refresh_token: 3e03e7aa7f9f4244928eb71a0100cfda1095d215


In [4]:
#' TODO:
#' >> get missing activities
#' >> routes?
#' >> explore: activity/route/segment_effort/segment streams

#' NOTE:
#' >> not need gear and clubs (as is already in athlete info)
#' >> not need activity laps (as is already in activity info)
#' >> only maybe need comments and kudos as count already in activites

## Get Athlete

In [5]:
# api-endpoint
athlete_url = "http://www.strava.com/api/v3/athlete/"
 
# defining a params dict for the parameters to be sent to the API
params = {'access_token':access_token}
 
# sending get request and saving the response as response object
r = requests.get(url = athlete_url, params = params)
 
# extracting data in json format
athlete = r.json()

print(athlete)
print(len(athlete))


############################
# create table clubs and add athlete_id as column
clubs = pd.DataFrame(athlete['clubs'])
clubs['athlete_id'] = athlete['id']

clubs.to_csv('clubs.csv')


############################
# create table gear and add athlete_id as column (add gear_type column)

bikes = pd.DataFrame(athlete['bikes'])
bikes['athlete_id'] = athlete['id']
bikes['gear_type'] = 'bike'

shoes = pd.DataFrame(athlete['shoes'])
shoes['athlete_id'] = athlete['id']
shoes['gear_type'] = 'shoe'

gear = pd.concat([bikes, shoes], ignore_index=True)

gear.to_csv('gear.csv')


############################
# create athlete DF

# Keys to drop
keys_to_drop = ['shoes', 'bikes', 'clubs']

# Remove the specified keys from the dictionary
for key in keys_to_drop:
    if key in athlete:
        del athlete[key]

athlete_df = pd.DataFrame(athlete, index = ['0'])
athlete_df.to_csv('athlete.csv')

{'id': 81994109, 'username': None, 'resource_state': 3, 'firstname': 'Michel', 'lastname': 'Meyer', 'bio': '', 'city': 'København', 'state': 'Capital Region of Denmark', 'country': 'Denmark', 'sex': 'M', 'premium': True, 'summit': True, 'created_at': '2021-03-28T16:59:10Z', 'updated_at': '2023-10-31T07:43:54Z', 'badge_type_id': 1, 'weight': 74.0, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/81994109/22810748/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/81994109/22810748/1/large.jpg', 'friend': None, 'follower': None, 'blocked': False, 'can_follow': True, 'follower_count': 29, 'friend_count': 31, 'mutual_friend_count': 0, 'athlete_type': 1, 'date_preference': '%m/%d/%Y', 'measurement_preference': 'meters', 'clubs': [{'id': 60197, 'resource_state': 2, 'name': 'Strava Runners Berlin', 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/clubs/60197/1319503/1/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfr

In [70]:
#with open("athlete.json", "w") as outfile:
#    json.dump(athlete, outfile)

## Get Athlete Zones

In [71]:
# api-endpoint
athlete_zones_url = "http://www.strava.com/api/v3/athlete/zones"
 
# defining a params dict for the parameters to be sent to the API
params = {'access_token':access_token}
 
# sending get request and saving the response as response object
r = requests.get(url = athlete_zones_url, params = params)
 
# extracting data in json format
athlete_zones = r.json()

print(athlete_zones)


############################
# Create DF for athlete zones
zones = []

for t in athlete_zones['heart_rate']['zones']:
    zones_row = []
    zones_row.append('heart_rate')
    zones_row.append(t['min'])
    zones_row.append(t['max'])
    zones.append(zones_row)

for t in athlete_zones['power']['zones']:
    zones_row = []
    zones_row.append('power')
    zones_row.append(t['min'])
    zones_row.append(t['max'])
    zones.append(zones_row)

athlete_zones = pd.DataFrame(zones)
athlete_zones.columns = ['type','min','max']
athlete_zones['athlete_id'] = athlete['id']

athlete_zones.to_csv('athlete_zones.csv')

{'heart_rate': {'custom_zones': False, 'zones': [{'min': 0, 'max': 127}, {'min': 127, 'max': 158}, {'min': 158, 'max': 174}, {'min': 174, 'max': 190}, {'min': 190, 'max': -1}]}, 'power': {'zones': [{'min': 0, 'max': 149}, {'min': 150, 'max': 203}, {'min': 204, 'max': 243}, {'min': 244, 'max': 284}, {'min': 285, 'max': 324}, {'min': 325, 'max': 405}, {'min': 406, 'max': -1}]}}


## Get Athlete Stats

In [96]:
# api-endpoint
athlete_stats_url = "http://www.strava.com/api/v3/athletes/" + str(athlete['id']) + "/stats"
 
# defining a params dict for the parameters to be sent to the API
params = {'access_token':access_token}
 
# sending get request and saving the response as response object
r = requests.get(url = athlete_stats_url, params = params)
 
# extracting data in json format
athlete_stats = r.json()

print(athlete_stats)


############################
# remove two stats (as they are of different format)
biggest_ride_distance = athlete_stats['biggest_ride_distance']
biggest_climb_elevation_gain = athlete_stats['biggest_climb_elevation_gain']

# Keys to drop
keys_to_drop = ['biggest_ride_distance', 'biggest_climb_elevation_gain',]

# Remove the specified keys from the dictionary
for key in keys_to_drop:
    if key in athlete_stats:
        del athlete_stats[key]

# restructure data format, so it fits in a wide-form DF
athlete_stats_array = []

for x in athlete_stats:
    for z in athlete_stats[x]:
        row = []
        row.append(x)
        row.append(z)
        row.append(athlete_stats[x][z])
        athlete_stats_array.append(row)

# create DF based on API output
athlete_stats_df = pd.DataFrame(athlete_stats_array)

# name columns of DF
athlete_stats_df.columns = ['group','key','value']

# add two rows that have been extracted before
athlete_stats_df.loc[len(athlete_stats_df)] = ['best_bike_performance', 'biggest_ride_distance', biggest_ride_distance]
athlete_stats_df.loc[len(athlete_stats_df)] = ['best_bike_performance', 'biggest_climb_elevation_gain', biggest_climb_elevation_gain]

athlete_stats_df['athlete_id'] = athlete['id']

athlete_stats_df.to_csv('athlete_stats.csv')

{'biggest_ride_distance': 101638.0, 'biggest_climb_elevation_gain': 112.315, 'recent_ride_totals': {'count': 1, 'distance': 63397.3203125, 'moving_time': 7005, 'elapsed_time': 7325, 'elevation_gain': 216.0, 'achievement_count': 95}, 'all_ride_totals': {'count': 142, 'distance': 5929612, 'moving_time': 765814, 'elapsed_time': 856115, 'elevation_gain': 24767}, 'recent_run_totals': {'count': 12, 'distance': 166465.931640625, 'moving_time': 47281, 'elapsed_time': 50426, 'elevation_gain': 645.8572616577148, 'achievement_count': 60}, 'all_run_totals': {'count': 430, 'distance': 5003856, 'moving_time': 1475923, 'elapsed_time': 1621735, 'elevation_gain': 22964}, 'recent_swim_totals': {'count': 4, 'distance': 9425.0, 'moving_time': 11385, 'elapsed_time': 14056, 'elevation_gain': 0.0, 'achievement_count': 0}, 'all_swim_totals': {'count': 73, 'distance': 134632, 'moving_time': 168198, 'elapsed_time': 208653, 'elevation_gain': 12}, 'ytd_ride_totals': {'count': 72, 'distance': 3265325, 'moving_time

In [48]:
# with open("athlete_stats.json", "w") as outfile:
#     json.dump(athlete_stats, outfile)

## Get Activities

In [30]:
# api-endpoint
activities_url = "http://www.strava.com/api/v3/athlete/activities"
 
# defining a params dict for the parameters to be sent to the API
params = {'access_token':access_token, 'per_page':'200', 'before':'1637884800'} 
# 'before':'1681776000'
# 'before':'1664640603'
# 'before':'1637884800'
 
# sending get request and saving the response as response object
r = requests.get(url = activities_url, params = params)
 
# extracting data in json format
activities = r.json()

# get activity-ids
activity_ids = [w['id'] for w in activities]

# print data structure
print(len(activities))
print(len(activities[0]))
print(activities[0])

110
48
{'resource_state': 2, 'athlete': {'id': 81994109, 'resource_state': 1}, 'name': 'Evening Run', 'distance': 10009.0, 'moving_time': 2904, 'elapsed_time': 3112, 'total_elevation_gain': 100.0, 'type': 'Run', 'sport_type': 'Run', 'workout_type': 0, 'id': 6302136410, 'start_date': '2021-11-24T17:12:16Z', 'start_date_local': '2021-11-24T18:12:16Z', 'timezone': '(GMT+01:00) Europe/Rome', 'utc_offset': 3600.0, 'location_city': None, 'location_state': None, 'location_country': None, 'achievement_count': 3, 'kudos_count': 1, 'comment_count': 1, 'athlete_count': 1, 'photo_count': 0, 'map': {'id': 'a6302136410', 'summary_polyline': 'cpw~Fa}gkALJ\\h@DT?d@GL[Nc@^}@lAKd@Cd@Hd@X|@RZ^f@`@vA`@x@J^AHQVoAfA[\\I^Gr@g@~BKn@KVW^]n@Sz@KTMt@Ul@]f@]VYFe@@WHgAz@o@`@_Av@k@\\GLQPm@b@OV_@Xe@Vw@La@PKNC\\CHKHYz@IFEGDDADGL{ApA}@lA}@x@_AbAs@l@k@l@Y^g@dAWn@MNaAb@c@Ja@A_@]iAmAw@]iB_BU[mA}@qAkA]UGO?IHo@z@kBz@{AXm@Hc@CWWi@{@qAUWYGa@Ak@Rk@d@a@h@O\\Y|@i@vAy@rA{@dBa@l@GPEZ@`@Bh@n@jDNzAPhAJ\\fArCR\\`@\\DPGl@iB|EiAnCWjAU

## Get Activity

In [154]:
activity_id = activity_ids[0]

# api-endpoint
activity_url = "http://www.strava.com/api/v3/activities/" + str(activity_id)
 
# defining a params dict for the parameters to be sent to the API
params = {'access_token':access_token}
 
# sending get request and saving the response as response object
r = requests.get(url = activity_url, params = params)
 
# extracting data in json format
activity = r.json()

print(activity)
print(len(activity))
print(activity.keys())
print(len(activity['segment_efforts']))

{'resource_state': 3, 'athlete': {'id': 81994109, 'resource_state': 1}, 'name': 'Afternoon Run', 'distance': 15596.1, 'moving_time': 4661, 'elapsed_time': 4844, 'total_elevation_gain': 33.0, 'type': 'Run', 'sport_type': 'Run', 'workout_type': None, 'id': 10126705515, 'start_date': '2023-10-29T15:46:56Z', 'start_date_local': '2023-10-29T16:46:56Z', 'timezone': '(GMT+01:00) Europe/Copenhagen', 'utc_offset': 3600.0, 'location_city': None, 'location_state': None, 'location_country': 'Denmark', 'achievement_count': 7, 'kudos_count': 4, 'comment_count': 0, 'athlete_count': 1, 'photo_count': 0, 'map': {'id': 'a10126705515', 'polyline': 'ueqrI{cxkAGCbAJfAfAp@lAJbBJl@PbEVnBCz@Ex@i@tCY|Bk@lDa@`DKrAAx@@PFHrAV`Cn@dANd@TNZLC\\RJPDNCTI^Uj@IJw@jBWbAy@hBUb@]`Ae@r@Kn@EBy@AYb@]x@Mh@DL`@T`@PRFZBn@?j@GxBNf@BREb@A^Lf@APG^Hp@@tAF`BPrAFnALt@@bBN|@^bCv@b@Jh@Rv@Lp@VlBf@jC|@BDB\\E|PArAOrDB|@HlA@t@GhTIdF@dBHvDAfHBj@NvA|AnIBd@A`@Ej@Kf@Yj@u@z@m@dAaBpDc@v@iAdCMr@Hv@^~@n@lBz@`BtBjCVPn@bAb@bARlALhBRvAVlF^vDl@jDh@zAv@

In [31]:
agg_segment_efforts = pd.DataFrame()
agg_segments = pd.DataFrame()
agg_activity_splits = pd.DataFrame()
agg_laps = pd.DataFrame()
agg_best_efforts = pd.DataFrame()
agg_activity = pd.DataFrame()

# loop through activities
for a in activity_ids:
    #print(a)

    # api-endpoint
    activity_url = "http://www.strava.com/api/v3/activities/" + str(a)
    # defining a params dict for the parameters to be sent to the API
    params = {'access_token':access_token}
    # sending get request and saving the response as response object
    r = requests.get(url = activity_url, params = params)
    # extracting data in json format
    activity = r.json()

    ############################
    if not activity['segment_efforts'] == []:
        # get segment efforts and segments from activity
        segment_efforts = pd.DataFrame(activity['segment_efforts'])
        # add columns to allow joining tables
        segment_efforts['activity_id'] = a
        segment_efforts['athlete_id'] = athlete['id']
        # drop unnecessary columns
        segment_efforts = segment_efforts.drop(columns = ['activity','athlete','achievements'])
        # create extra df for segments
        segments = pd.DataFrame(segment_efforts['segment'].tolist())
        # rejoin segment_id onto segment effort
        segment_ids = []
        for o in segment_efforts['segment']:
            segment_ids.append(o['id'])
        segment_efforts['segment_id'] = segment_ids
        # join segment efforts
        agg_segment_efforts = pd.concat([agg_segment_efforts,segment_efforts], axis = 0)
        # join segments
        agg_segments = pd.concat([agg_segments,segments], axis = 0)

    ############################
    # get metric splits
    if 'splits_metric' in activity.keys():
        activity_splits = pd.DataFrame(activity['splits_metric'])
        #print(activity_splits.columns)
        #print(len(activity_splits))
        # add columns to allow joining tables
        activity_splits['activity_id'] = a
        activity_splits['athlete_id'] = athlete['id']
        # drop na columns
        activity_splits = activity_splits.dropna(axis=1, how='all')
        activity_splits = activity_splits.dropna(axis=0, how='all')
        if 'average_grade_adjusted_speed' in activity_splits.columns:
            activity_splits = activity_splits.drop(columns = ['average_grade_adjusted_speed'])
        # join activity splits
        agg_activity_splits = pd.concat([agg_activity_splits,activity_splits], axis = 0)

    ############################
    if 'laps' in activity.keys():
        # get lap data from activity
        laps = pd.DataFrame(activity['laps'])
        # add columns to allow joining tables
        laps['activity_id'] = a
        laps['athlete_id'] = athlete['id']
        laps = laps.drop(columns = ['activity','athlete'])
        # join laps
        agg_laps = pd.concat([agg_laps,laps], axis = 0)

    ############################
    try:
        if 'best_efforts' in activity.keys():
            # get best efforts from activity
            best_efforts = pd.DataFrame(activity['best_efforts'])
            # add columns to allow joining tables
            best_efforts['activity_id'] = a
            best_efforts['athlete_id'] = athlete['id']
            best_efforts = best_efforts.drop(columns = ['activity','athlete','achievements'])
            # join best efforts
            agg_best_efforts = pd.concat([agg_best_efforts,best_efforts], axis = 0)
    except:
        print('else')

    ############################
    # Get actual activity data
    keys_to_drop = ['athlete','map','segment_efforts','splits_metric','splits_standard','laps','best_efforts','stats_visibility','photos','similar_activities','gear','available_zones','start_latlng','end_latlng']
    # Remove the specified keys from the dictionary
    for key in keys_to_drop:
        if key in activity:
            del activity[key]
    # create DF and merge with other activities
    activity_df = pd.DataFrame(activity, index = [0])
    activity_df['athlete_id'] = athlete['id']
    agg_activity = pd.concat([agg_activity,activity_df])


# create csv for segment efforts
agg_segment_efforts.to_csv('segment_efforts4.csv')
# create csv for unique segments
unique_segments = agg_segments.drop_duplicates(subset=['id'])
unique_segments.to_csv('segments4.csv')
# create csv for splits
agg_activity_splits.to_csv('activity_splits4.csv')
# create csv for laps
agg_laps.to_csv('laps4.csv')
# create csv for best efforts
agg_best_efforts.to_csv('best_efforts4.csv')
# create csv for activity
agg_activity.to_csv('activity4.csv')

else


/tmp/ipykernel_14874/2218960912.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  agg_segment_efforts = pd.concat([agg_segment_efforts,segment_efforts], axis = 0)
/tmp/ipykernel_14874/2218960912.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  agg_segment_efforts = pd.concat([agg_segment_efforts,segment_efforts], axis = 0)
/tmp/ipykernel_14874/2218960912.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will

else


/tmp/ipykernel_14874/2218960912.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  agg_activity = pd.concat([agg_activity,activity_df])


else


...

else


/tmp/ipykernel_14874/2218960912.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  agg_activity = pd.concat([agg_activity,activity_df])


else


/tmp/ipykernel_14874/2218960912.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  agg_activity = pd.concat([agg_activity,activity_df])


else


### Get Activity Zones

In [32]:
agg_athlete_zones = pd.DataFrame()

for a in activity_ids:
    print(a)
    # api-endpoint
    activity_zone_url = "http://www.strava.com/api/v3/activities/" + str(a) + "/zones"
    # defining a params dict for the parameters to be sent to the API
    params = {'access_token':access_token}
    # sending get request and saving the response as response object
    r = requests.get(url = activity_zone_url, params = params)
    # extracting data in json format
    activity_zones = r.json()

    zones = []
    for t in activity_zones:
        if 'score' in t.keys():
            for r in t['distribution_buckets']:
                zones_row = []
                zones_row.append(r['min'])
                zones_row.append(r['max'])
                zones_row.append(r['time'])
                zones_row.append(t['type'])
                zones_row.append(t['resource_state'])
                zones_row.append(t['sensor_based'])
                zones_row.append(t['score'])
                zones_row.append(athlete['id'])
                zones_row.append(a)
                zones.append(zones_row)

    athlete_zones = pd.DataFrame(zones)
    agg_athlete_zones = pd.concat([agg_athlete_zones,athlete_zones], axis = 0)

agg_athlete_zones.columns = ['min','max','time','type','resource_state','sensor_based','score','athlete_id','activity_id']
agg_athlete_zones.to_csv('activity_zones4.csv')

6302136410


KeyboardInterrupt: 